In [57]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from random import shuffle
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense


Using TensorFlow backend.


In [66]:
fer_path = '../Data/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013/fer2013.csv'
data = pd.read_csv(fer_path)

In [67]:
np.unique(data['Usage'], return_counts=True)

(array(['PrivateTest', 'PublicTest', 'Training'], dtype=object),
 array([ 3589,  3589, 28709]))

In [69]:
training_pixels = data[data['Usage'] == 'Training']['pixels'].tolist()
width, height = 48, 48
image_size = (width, height)
X_train = []
for pixel_sequence in training_pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    #     face = cv2.resize(face.astype('uint8'), image_size)
    X_train.append(face.astype('float32'))
    
X_train = np.asarray(X_train)
X_train = np.expand_dims(X_train, -1)
y_train = pd.get_dummies(data[data['Usage'] == 'Training']['emotion']).values

In [70]:
training_pixels = data[data['Usage'] == 'PublicTest']['pixels'].tolist()
width, height = 48, 48
image_size = (width, height)
X_test = []
for pixel_sequence in training_pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    #     face = cv2.resize(face.astype('uint8'), image_size)
    X_test.append(face.astype('float32'))
X_test = np.asarray(X_train)

X_test = np.expand_dims(X_test, -1)
y_test = pd.get_dummies(data[data['Usage'] == 'PublicTest']['emotion']).values

Faces shape type is (#records, image_width, image_height, 1)

In [74]:
IMG_SIZE = 48
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
# model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(7, activation = 'softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=100, epochs=20, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
28709/28709 [==============================] - 43s 1ms/step - loss: 1.6119 - acc: 0.3958
Epoch 2/20
12700/28709 [============>.................] - ETA: 22s - loss: 1.3040 - acc: 0.5021

In [73]:
y_train.shape

(28709, 7)